##### Notebook com a resolução do teste técnico - Atmo -<br>


In [695]:
!pip3 install mysql-connector-python

In [696]:
import mysql.connector
import pandas as pd
import warnings
import re

warnings.filterwarnings('ignore')

1)	Escreva um script em python que consuma cada uma das tabelas no schema e transforme-as em dataframes, ao transforma-los podemos notar que as colunas dos dataframes estão nomeadas no padrão camel case, passe-as para snake case.

In [697]:
# Está célula contém o código necessário para a realização da primeira questão do teste

def get_data_table(table_lobby, table_players_med, table_players):
    """Função que executa a query no banco de dados

        Parameters:
            table_lobby: tabela de dados das partidas de CS
            table_players_med:  tabela de dados das medalhas
            table_players:  tabela de dados dos jogadores

        Returns:
            Todas as tabelas em formato dataframe.
    """  
    

    conn = mysql.connector.connect(host='atmo-db.cncfsgdjnfjz.sa-east-1.rds.amazonaws.com',database='CSGO',user='candidato',password='analista_atmo')

    query_lobby = f"SELECT * FROM CSGO.{table_lobby};"
    query_players_med = f"SELECT * FROM CSGO.{table_players_med};"
    query_players = f"SELECT * FROM CSGO.{table_players};"

    table_lobby = pd.read_sql(query_lobby, conn)
    table_players_med = pd.read_sql(query_players_med, conn)
    table_players = pd.read_sql(query_players, conn)

    conn.close()

    return table_lobby, table_players_med, table_players


def rename_pattern(df):
    """Função para converte o padrão de nomeclatura camel case para snake case

        Parameters:
            df: dataframe que será manipulado 

        Returns:
            Dataframe no padrão snake case
    """  
    
    lista = list(df)

    for i in lista:
            df.rename(columns = {i : re.sub('[A-Z]', lambda _ : '_' +  _.group(0).lower(), i)}, inplace= True)
    return df


def get_data():
    """Função principal que faz a chamada ao banco de dados e retorna tabelas já no padrão snake case.
        Returns:
            Todas as tabelas em formato dataframe já padronizada.
    """  
    get_tables = get_data_table('tb_lobby_stats_player','tb_players_medalha', 'tb_players')

    df_lobby = rename_pattern(get_tables[0])
    df_players_med = rename_pattern(get_tables[1])
    df_players = rename_pattern(get_tables[2])

    return df_lobby, df_players_med, df_players

df_lobby = get_data()[0]
df_players_med = get_data()[1]
df_players = get_data()[2]


2. Faça uma query em SQL que retorne uma tabela com Id do player, taxa de vitória e número de partidas dos jogadores que tiverem acima de 10 partidas, selecione os jogadores com as 50 maiores taxas de vitória.

In [698]:
# Está célula contém o código necessário para a realização da segunda questão do teste
query_statitics = f"""WITH get_statitics_players AS (
SELECT  
        idPlayer,
        SUM(qtKill)         AS sum_qtKill,
        SUM(qtAssist)       AS sum_qtAssist,
        SUM(qtDeath)        AS sum_qtDeath,
        SUM(qtHs)           AS sum_qtHs,
        SUM(qtRoundsPlayed) AS sum_qt_rounds_played,
        SUM(qtSurvived)     AS sum_qt_survived,
        SUM(flWinner)       AS sum_fl_winner,
        COUNT(idPlayer)     AS qtMatches           

FROM CSGO.tb_lobby_stats_player
GROUP BY idPlayer),

create_txWinner  AS (

SELECT  
        idPlayer,
        CAST(sum_qtKill AS SIGNED)            AS qtKill,
        CAST(sum_qtAssist AS SIGNED)          AS qtAssist,
        CAST(sum_qtDeath AS SIGNED)           AS qtDeath,
        CAST(sum_qtHs AS SIGNED)              AS qtHs,
        CAST(sum_qt_rounds_played AS SIGNED)  AS qtRoundsPlayed, 
        CAST(sum_qt_survived AS SIGNED)       AS qtSurvived,
        CAST(sum_fl_winner AS SIGNED)         AS flWinner,
        qtMatches,
        CAST(sum_fl_winner / qtMatches  * 100 AS DECIMAL(8,2))    AS txWinner

FROM get_statitics_players
WHERE qtMatches > 10)

SELECT 
        *
FROM create_txWinner 
ORDER BY txWinner DESC
LIMIT 50
"""
conn = mysql.connector.connect(host='atmo-db.cncfsgdjnfjz.sa-east-1.rds.amazonaws.com', database='CSGO', user='candidato', password='analista_atmo')


df_statitics = pd.read_sql(query_statitics, conn)
df_statitics

,idPlayer,qtKill,qtAssist,qtDeath,qtHs,qtRoundsPlayed,qtSurvived,flWinner,qtMatches,txWinner
0,1237,371,72,286,162,474,182,17,18,94.44
1,397,233,44,221,105,348,108,11,13,84.62
2,1262,262,54,220,96,325,98,11,13,84.62
3,306,351,66,319,143,468,138,15,18,83.33
4,1083,426,107,334,144,488,143,15,18,83.33
5,2120,275,62,211,93,312,93,10,12,83.33
6,2545,455,67,271,230,444,160,14,17,82.35
7,2054,364,53,327,172,480,152,14,17,82.35
8,2311,198,49,211,83,299,82,9,11,81.82
9,459,290,45,253,140,395,133,13,16,81.25


3. Uma das métricas para medir o desempenho do jogador é o rating “K/D” que consiste no número de eliminações realizadas dividido pelo número de vezes que o jogador foi eliminado, Faça uma query em SQL e selecione o id do player, país de origem, idade e rating kd.

In [699]:
# Está célula contém o código necessário para a realização da terceira questão do teste

query_statitics_kd = f"""WITH get_statitics_players AS (
SELECT  
        tb_stats_player.idPlayer,
        SUM(tb_stats_player.qtKill)         AS sum_qtKill,
        SUM(tb_stats_player.qtAssist)       AS sum_qtAssist,
        SUM(tb_stats_player.qtDeath)        AS sum_qtDeath,
        tb_players.descCountry,
        DATEDIFF(CURDATE(), DATE_FORMAT(DATE(tb_players.dtBirth), '%Y-%m-%d'))     AS agePlayer


FROM CSGO.tb_lobby_stats_player AS tb_stats_player
INNER JOIN CSGO.tb_players      AS tb_players
ON tb_stats_player.idPlayer = tb_players.idPlayer
GROUP BY tb_stats_player.idPlayer),

date_format AS (

SELECT  
        idPlayer,
        (CAST(sum_qtKill AS SIGNED) / CAST(sum_qtDeath AS SIGNED)) AS ratingKD,
        descCountry,
        CAST(CAST(agePlayer AS SIGNED) / 360 AS SIGNED)  AS  agePlayer

FROM get_statitics_players)

SELECT 
         idPlayer,
         descCountry,
         CAST(agePlayer AS SIGNED INTEGER) AS agePlayer,
         ratingKD
         
FROM date_format

"""
conn = mysql.connector.connect(host='atmo-db.cncfsgdjnfjz.sa-east-1.rds.amazonaws.com',database='CSGO',user='candidato',password='analista_atmo')


df_statitics_kd = pd.read_sql(query_statitics_kd, conn)
df_statitics_kd

,idPlayer,descCountry,agePlayer,ratingKD
0,1,cl,NaN,0.8529
1,2,br,NaN,0.4823
2,3,br,NaN,1.0995
3,4,br,NaN,0.6591
4,5,ar,NaN,1.1429
...,...,...,...,...
2464,2712,br,23.0,1.0106
2465,2713,br,NaN,0.6500
2466,2714,br,23.0,0.9327
2467,2715,br,22.0,0.5135


Pokémon é uma série de jogos eletrônicos desenvolvidos pela Game Freak e publicados pela Nintendo como parte da franquia de mídia Pokémon. Lançado pela primeira vez em 1996 no Japão para o console Game Boy, a principal série de jogos de RPGs, que continuou em cada geração em portáteis.

4)	Utilize a API de referência para buscar informações sobre os pokemons da primeira geração (150) e monte um dataframe contendo seus nomes, suas habilidades, um dos locais onde ele pode ser encontrado, tipo, hp, ataque e defesa.
Descreva qual seria a forma mais interessante para requisitar esses dados da API caso o número de pokemons fosse na verdade 1.500.000. Caso deseje, desenvolva o código já com essa mentalidade.

Link API: https://pokeapi.co/docs/v2


In [700]:
# Está célula contém o código necessário para a realização da quarta questão do teste

from requests.exceptions import HTTPError
import requests
import json

def cleaning_df(df_pokemon_results):
    """Função que faz a limpeza do dataframe, alterando nome das colunas, excluindo colunas e registros duplicados.

        Parameters:
            df_pokemon_results: dataframe que será manipulado

        Returns:
            Dataframe com padronização no nome das colunas e sem dados duplicados
    """  

    df_pokemon_results.rename(columns = {'name_x': 'name'}, inplace = True)
    df_pokemon_results.drop(['is_hidden', 'slot_x', 'slot_y','name_y', 'ability_url'], axis = 1, inplace= True)
    columns = ['id', 'name', 'ability_name', 'type_name', 'location_area_encounters']
    df_pokemon_results = df_pokemon_results[columns]
    df_pokemon_results.drop_duplicates()

    return df_pokemon_results
    


def json_normalize(df_json, record_path = [], meta = []):
    """Função que faz a normalização dos objetos do tipo json

        Parameters:
            df_json: dataframe retornado na chamada a api
            record_path: caminho/nível dos registros que serão considerados a nível hierárquico 
            meta = Campos a serem usados como metadados, colunas que serão tabularizadas

        Returns:
            Dataframe normalizado com as colunas escolhidas no parametro meta
        """

    df = pd.json_normalize(df_json, record_path = record_path, meta = meta, errors= 'ignore', sep ='_')

    return df

def main(endpoint_one, endpoint_two):
    """Função que irá faz a chamada a api

        Parameters:
           endpoint_one: parâmetro que será passado no primeiro endpoint, para resolução do problemao endpoint é a geração
           endpoint_two: parâmetro que será passado no segundo endpoint, para resolução do problema é o número da geração

        Returns:
            Dataframe já tratado depois da chamada a api.
    """
    

    try:
        api = f'https://pokeapi.co/api/v2/{endpoint_one}/{endpoint_two}'
        response = requests.get(api, verify=False)
        if response.status_code != 200:
            print(f'Error: {response.json().get(("errorMessage"))}')
        else:
            pokemons_name = []
            poke_list = response.json()
            for i in poke_list['pokemon_species']:
                pokemons_name.append(i['name'])

            df = pd.DataFrame()
            df_auxiliar_type = pd.DataFrame()

            for j in pokemons_name:
                url_api = f'https://pokeapi.co/api/v2/pokemon/{j}'
                resp_api = requests.get(url_api, verify=False)
                resp_json = resp_api.json()
                
                df_abilities_data = json_normalize(resp_json, record_path = ['abilities'],  meta =['name', 'location_area_encounters', 'id'])
                df_type_data = json_normalize(resp_json, record_path = ['types'], meta =['name', 'id'])
      

                df = pd.concat([df, df_abilities_data])
                df_auxiliar_type = pd.concat([df_auxiliar_type, df_type_data])

                df_pokemon_results = df.merge(df_auxiliar_type, on = 'id')
          

                
    except HTTPError as http_err:
        print(f'HTTP error: {http_err:}')

    except Exception as error:
        print(f'Error: {error}')
    else:
        df_pokemon_results = cleaning_df(df_pokemon_results)
        print('Sucess')

    return df_pokemon_results


df_pokemon_results = main('generation', '1')
df_pokemon_results


Sucess


In [713]:
df_statitics_kd

,idPlayer,descCountry,agePlayer,ratingKD
0,1,cl,NaN,0.8529
1,2,br,NaN,0.4823
2,3,br,NaN,1.0995
3,4,br,NaN,0.6591
4,5,ar,NaN,1.1429
...,...,...,...,...
2464,2712,br,23.0,1.0106
2465,2713,br,NaN,0.6500
2466,2714,br,23.0,0.9327
2467,2715,br,22.0,0.5135


In [735]:
import plotly.express as px
df = df_statitics_kd
fig = px.bar(df, x='descCountry', y='ratingKD', color ='descCountry', color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(title_text='RatingKD por País', title_x=0.5) 

fig.show()
